In [10]:
!huggingface-cli login --token $TOKEN$ --add-to-git-credential 

/bin/bash: /home/yashc/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/yashc/.cache/huggingface/token
Login successful


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset, Dataset

dataset = load_dataset("mlsquare/samantar_merged_with_train_val")
tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#DATA COMBINATION

In [2]:
# Import necessary libraries
from datasets import load_dataset, concatenate_datasets, DatasetDict  # For loading datasets and concatenating them
from tqdm import tqdm  # For progress bar visualization
from collections import defaultdict  # For creating a dictionary with default values

# List of language codes for datasets
language_codes = ['as', 'bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'or', 'pa', 'ta', 'te']

# Empty list to store datasets
datasets_list = []

# Loop through each language code
for code in tqdm(language_codes):
    # Load dataset for each language code
    dataset = load_dataset("ai4bharat/samanantar", code)
    # Append only the training portion of the dataset to the list
    datasets_list.append(dataset["train"])

# Concatenate all the datasets into a single dataset
merged_dataset = concatenate_datasets(datasets_list)

# Function to filter concatenated dataset based on a limit per language
def filter_concatenated_dataset(merged_dataset, limit_per_language=5000):
    # Dictionary to store filtered data
    filtered_dict = defaultdict(list)
    total = 0  # Counter for total number of samples processed
    
    # Loop through each sample in the merged dataset
    for sample in tqdm(merged_dataset):
        # Check if the total number of samples processed is less than the limit per language times the number of languages
        if total < limit_per_language * len(language_codes):
            # Append the target and source texts to the filtered dictionary
            filtered_dict["tgt"].append(sample["tgt"])
            filtered_dict["src"].append(sample["src"])
            total += 1  # Increment the counter
        else:
            break  # If the limit is reached, exit the loop
            
    # Calculate the percentage of data retained after filtering
    print(f"{len(filtered_dict['tgt'])/total:.2%} of data after filtering.")
    
    # Return the filtered dataset as a Dataset object
    return Dataset.from_dict({"tgt": filtered_dict["tgt"], "src": filtered_dict["src"]})


  0%|                                                                 | 0/11 [00:00<?, ?it/s]/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|████████████████████████████████████████████████████████| 11/11 [00:47<00:00,  4.33s/it]


In [3]:
filtered_merged_dataset = filter_concatenated_dataset(merged_dataset,limit_per_language=10000000)


100%|█████████████████████████████████████████| 49774246/49774246 [19:41<00:00, 42121.60it/s]


200.00% of data after filtering.


In [4]:
# Determine the number of samples for train and validation sets
from datasets import Dataset, DatasetDict
train_size = int(len(filtered_merged_dataset) * 0.8)  # 80% for training
valid_size = len(filtered_merged_dataset) - train_size  # Remaining for validation

# Split the dataset into train and validation sets
ds_train = filtered_merged_dataset.select(list(range(train_size)))
ds_valid = filtered_merged_dataset.select(list(range(train_size, train_size + valid_size)))

# Create DatasetDict with train and validation sets
raw_datasets = DatasetDict({"train": ds_train, "valid": ds_valid})

In [5]:
raw_datasets.push_to_hub("mlsquare/samantar_merged_with_train_val")

Pushing dataset shards to the dataset hub:   0%|          | 0/22 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3620 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3620 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3620 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3620 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/3982 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3982 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3982 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3982 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3982 [00:00<?, ?ba/s]